<a href="https://colab.research.google.com/github/KalanaKariyawasam/RectCoGen-Machine-Learning-Based-Code-Generation-for-Front-end-implementation/blob/main/RectCoGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Kalana Kariyawasam  
w1809765  
2019353

#RectCoGen: Machine Learning Based Code Generation for Front-end Implementation

**Domain:** Front-end Development  
**Problem Overview:** The labor-intensive and error-prone process of manually converting web wireframes and graphical user interfaces (GUIs) into functional code. Traditionally, this task requires extensive collaboration between designers and developers, leading to increased development time, higher costs, and potential for discrepancies between the intended design and the implemented functionality. RectCoGen seeks to simplify this process by introducing an automated solution that transforms design blueprints into React components, aiming to improve efficiency, reduce errors, and lower financial burdens associated with web development.






